In [3]:
#environment variables
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [4]:
# Extract text from PDF
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

# provide the path of  pdf file/files.
pdfreader = PdfReader('7.pdf')

In [5]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

In [7]:
# We need to split the text using Character Text Split such that it should not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
print(texts)

In [ ]:

#langchain dependencies
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

#prompt template
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [("system", '''You are an AI bot designed to act as a professional for parsing resumes. You are given with resume and your job is to extract the following information from the resume:
    1. Full Name
    2. Email Id
    3. Phone
    4. Github Portfolio
    5. Linkedin Id
    6. Education Details
    7. Employment Details
    8. Technical Skills
    9. Soft Skills
    10. Certifications
    11. Publications
    Give the extracted information in json format only'''),
    ("user", "Content:{content}")
    ]
)
    
#OpenAI LLM
llm=ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()
chain=prompt|llm|output_parser

print (chain.invoke({'content': texts}))
